In [ ]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, desc

# from config import api_key

import numpy as np
import requests
import json

In [ ]:
# Import files as DataFrames
fourteen_fiftenn_pd = pd.read_csv("draft_data/2014-15.csv")
seventeen_eighteen_pd = pd.read_csv("draft_data/2017-18.csv", encoding = "ISO-8859-1")
twenty_twentyone = pd.read_csv("draft_data/2020-21.csv", encoding = "ISO-8859-1")

In [ ]:
# Rename columns to be uniform across dataframes
twenty_twentyone.rename(columns={"Facility name":"Facility Name"}, inplace=True)

In [ ]:
# Remove rows of corporate total data
fourteen_fiftenn_pd = fourteen_fiftenn_pd[fourteen_fiftenn_pd['Facility Name'] != 'Corporate Total']
seventeen_eighteen_pd = seventeen_eighteen_pd[seventeen_eighteen_pd['Facility Name'] != 'Corporate Total']
twenty_twentyone = twenty_twentyone[twenty_twentyone['Facility Name'] != 'Corporate Total']

In [ ]:
# Drop columns of un-needed data
fourteen_fiftenn_pd = fourteen_fiftenn_pd.drop(columns={"Type", "Scope 1 (t CO2-e)","Scope 2 (t CO2-e)","Total Emissions (t CO2-e)", "Emission  Intensity (t/Mwh)", "Grid Connected", "Grid", "Important Notes"})
seventeen_eighteen_pd = seventeen_eighteen_pd.drop(columns={"Type", "Total Scope 1 Emissions \n(t CO2-e)","Total Scope 2 Emissions \n(t CO2-e)","Total Emissions \n(t CO2-e)", "Emission Intensity \n(t CO2-e/ MWh)", "Grid Connected", "Grid"})
twenty_twentyone = twenty_twentyone.drop(columns={"Type", "Total scope 1 emissions \n(t CO2-e)","Total scope 2 emissions \n(t CO2-e)","Total emissions \n(t CO2-e)", "Emission intensity \n(t CO2-e/ MWh)", "Grid connected", "Grid"})


In [ ]:
# Add year column for filtering in plots
fourteen_fiftenn_pd['Year'] = '2014-2015'

seventeen_eighteen_pd['Year'] = '2017-2018'

twenty_twentyone['Year'] = '2020-2021'

In [ ]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in fourteen_fiftenn_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']
    state = row['State']

    # update address key value
    params['address'] = f"{location}+{state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        fourteen_fiftenn_pd.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        fourteen_fiftenn_pd.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        fourteen_fiftenn_pd.loc[index, "Lat"] = "NaN"
        fourteen_fiftenn_pd.loc[index, "Lng"] = "NaN"

# Visualize to confirm lat lng appear
fourteen_fiftenn_pd.head()

In [ ]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in seventeen_eighteen_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']
    state = row['State']

    # update address key value
    params['address'] = f"{location}+{state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        seventeen_eighteen_pd.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        seventeen_eighteen_pd.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        seventeen_eighteen_pd.loc[index, "Lat"] = "NaN"
        seventeen_eighteen_pd.loc[index, "Lng"] = "NaN"

seventeen_eighteen_pd.head()

In [ ]:
# create a params dict that will be updated with new city each iteration
params = {"key": api_key}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in twenty_twentyone.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    location = row['Facility Name']
    state = row['State']

    # update address key value
    params['address'] = f"{location} + {state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
#     # convert to json
    cities_lat_lng = cities_lat_lng.json()
    
    try:
        twenty_twentyone.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
        twenty_twentyone.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        twenty_twentyone.loc[index, "Lat"] = "NaN"
        twenty_twentyone.loc[index, "Lng"] = "NaN"

twenty_twentyone.head()

In [ ]:
# Drop rows which returned no coordinates for facilities
fourteen_fiftenn_pd = fourteen_fiftenn_pd[fourteen_fiftenn_pd['Lat'] != 'NaN']

seventeen_eighteen_pd = seventeen_eighteen_pd[seventeen_eighteen_pd['Lat'] != 'NaN']

twenty_twentyone = twenty_twentyone[twenty_twentyone['Lat'] != 'NaN']

In [ ]:
twenty_twentyone.head()

In [ ]:
# fourteen_fifteen, seventeen_eighteen, twenty_twentyone
# filter each df to group by state and then group by again to show primary fuel types

In [ ]:
# // Energy Tables: Titled 14_15_Energy_Production, 17_18_Energy_Production & 20_21_Energy_Production
# // Energy Tables Column Headers:
# // # Reporting entity
# // # Facility name
# // # State
# // # Electricity production (GJ)
# // # Electricity production (MWh)

# DELETE THESE COLUMNS
# // # Total scope 1 emissions (t CO2-e)
# // # Total scope 2 emissions (t CO2-e)
# // # Total emissions (t CO2-e)
# // # Emission intensity (t CO2-e/ MWh)
# // # Grid connected
# // # Grid

# // # Primary fuel
# Lat
# Long
# // # Year
# // # Renewable

# // Population Table Titled Aus_Population
# // Aus_Population Column Headers:
# // # State
# // # Male
# // # Female
# // # Total

# // Income Table Titled Aus_Income
# // Aus_Income Column Headers:
# // # State
# // # Median Income

In [ ]:
# fourteen_fifteen_cols = ["Reporting_Entity", "Facility_Name", "State", "Electricity_Production_GJ", "Electricity_Production_MWh","Primary_Fuel", "Lat", "Long", "Year"]
# fourteen_fifteen_transformed = fourteen_fiftenn_pd[fourteen_fifteen_cols].copy()

fourteen_fifteen_transformed = fourteen_fiftenn_pd.rename(columns={"Reporting Entity ": "Reporting_Entity",
                                                                   "Facility Name": "Facility_Name",
                                                                   "Electricity Production (GJ)": "Electricity_Production_GJ",
                                                                   "Electricity Production (Mwh)": "Electricity_Production_MWh",
                                                                   "Primary Fuel": "Primary_Fuel"})

In [ ]:
# seventeen_eighteen_cols = ["Reporting_Entity", "Facility_Name", "State", "Electricity_Production_GJ", "Electricity_Production_MWh","Primary_Fuel", "Lat", "Long", "Year"]
# seventeen_eighteen_transformed = seventeen_eighteen_pd[seventeen_eighteen_cols].copy()

seventeen_eighteen_transformed = seventeen_eighteen_pd.rename(columns={"Reporting Entity": "Reporting_Entity",
                                                                   "Facility Name": "Facility_Name",
                                                                   "Electricity Production (GJ)": "Electricity_Production_GJ",
                                                                   "Electricity Production (MWh)": "Electricity_Production_MWh",
                                                                   "Primary Fuel": "Primary_Fuel"})

In [ ]:
# seventeen_eighteen_cols = ["Reporting_Entity", "Facility_Name", "State", "Electricity_Production_GJ", "Electricity_Production_MWh","Primary_Fuel", "Lat", "Long", "Year"]
# seventeen_eighteen_transformed = seventeen_eighteen_pd[seventeen_eighteen_cols].copy()

twenty_twentyone_transformed = twenty_twentyone.rename(columns={"Reporting entity": "Reporting_Entity",
                                                                   "Facility Name": "Facility_Name",
                                                                   "Electricity production (GJ)": "Electricity_Production_GJ",
                                                                   "Electricity production (MWh)": "Electricity_Production_MWh",
                                                                   "Primary fuel": "Primary_Fuel"})

In [ ]:
renewables = ['Hydro', 'Landfill Gas', 'Wind', 'Solar', 'Bagasse', 'Biofuel', 'Macadamia Nut Shells', 'Biogas']

fourteen_fifteen_transformed['Renewable'] = fourteen_fifteen_transformed.Primary_Fuel.isin(renewables)
seventeen_eighteen_transformed['Renewable'] = seventeen_eighteen_transformed.Primary_Fuel.isin(renewables)
twenty_twentyone_transformed['Renewable'] = twenty_twentyone_transformed.Primary_Fuel.isin(renewables)

In [ ]:
fourteen_fifteen_transformed.drop(columns=['index'], inplace=True)

In [ ]:
twenty_twentyone_transformed.head()

In [ ]:
seventeen_eighteen_transformed.drop(columns=['index'], inplace=True)
twenty_twentyone_transformed.drop(columns=['index'], inplace=True)

In [ ]:
# import psycopg2

# connection_string = "postgres:postgres@localhost:5432/energy_db"
# engine = create_engine(f'postgresql://{connection_string}')

# # Confirm tables
# engine.table_names()

# fourteen_fifteen_transformed.to_sql(name='Fourteen_Fifteen_Energy_Production', con=engine, if_exists='append', index=False)
# seventeen_eighteen_transformed.to_sql(name='Seventeen_Eighteen_Energy_Production', con=engine, if_exists='append', index=False)
# twenty_twentyone_transformed.to_sql(name='Twenty_Twentyone_Energy_Production', con=engine, if_exists='append', index=False)

In [ ]:
fourteen_fifteen_transformed.to_csv('transformed_csvs/14-15.csv')
seventeen_eighteen_transformed.to_csv('transformed_csvs/17-18.csv')
twenty_twentyone_transformed.to_csv('transformed_csvs/20-21.csv')

In [55]:
# Import files as DataFrames
ff_transformed = pd.read_csv("transformed_csvs/14-15.csv")
se_transformed = pd.read_csv("transformed_csvs/17-18.csv")
tt_transformed = pd.read_csv("transformed_csvs/20-21.csv")

In [ ]:
import pymongo

# Setup connection to mongodb
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Create database and collection to use
db = client['energy_db']


In [ ]:
fourt_fift_energy_production = db.Fourteen_Fifteen_Energy_Production
sevt_eight_energy_production = db.Seventeen_Eighteen_Energy_Production
twe_twentyo_energy_production = db.Twenty_Twentyone_Energy_Production
aus_population = db.Aus_Population
aus_income = db.Aus_Income

In [ ]:
fourt_fift_energy_production.insert_many(ff_transformed.to_dict('records'))
sevt_eight_energy_production.insert_many(se_transformed.to_dict('records'))
twe_twentyo_energy_production.insert_many(tt_transformed.to_dict('records'))

visualisation that shows each state renewables growth? Ie, a double bar graph, showing the amount of renewable plants in WA in 2013 next to the amount of renewable plants in WA in 2020 Using Dataframes 14/15, 17/18 & 20/21.


In [ ]:
# ff_transformed
# se_transformed
# tt_transformed

In [56]:
ff_transformed_progression = ff_transformed[["State", "Year", "Renewable"]]
ff_transformed_progression.head(2)

,State,Year,Renewable
0,NSW,2014-2015,True
1,ACT,2014-2015,True


In [57]:
ff_transformed_progression_renewable = ff_transformed_progression.loc[ff_transformed_progression["Renewable"] == True]
ff_transformed_progression_renewable

,State,Year,Renewable
0,NSW,2014-2015,True
1,ACT,2014-2015,True
2,VIC,2014-2015,True
3,VIC,2014-2015,True
5,VIC,2014-2015,True
...,...,...,...
290,SA,2014-2015,True
291,SA,2014-2015,True
297,NSW,2014-2015,True
298,QLD,2014-2015,True


In [58]:
# group dataframe by State
ff_transformed_progression_renewable_grouped = ff_transformed_progression_renewable.groupby(["State"], as_index=False).count()
ff_transformed_progression_renewable_grouped

,State,Year,Renewable
0,ACT,4,4
1,NSW,38,38
2,NT,2,2
3,QLD,14,14
4,SA,10,10
5,TAS,26,26
6,VIC,37,37
7,WA,15,15


In [59]:
# format DF
ff_transformed_progression_renewable_grouped = ff_transformed_progression_renewable_grouped[["State", "Renewable"]]
ff_transformed_progression_renewable_grouped.rename(columns={'Renewable': 'Renewable_Energy_Plants_2014_2015'}, inplace=True)
ff_transformed_progression_renewable_grouped

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,State,Renewable_Energy_Plants_2014_2015
0,ACT,4
1,NSW,38
2,NT,2
3,QLD,14
4,SA,10
5,TAS,26
6,VIC,37
7,WA,15


In [60]:
ff_transformed_progression_nonrenewable = ff_transformed_progression.loc[ff_transformed_progression["Renewable"] == False]
# ff_transformed_progression_nonrenewable

# group dataframe by State
ff_transformed_progression_nonrenewable_grouped = ff_transformed_progression_nonrenewable.groupby(["State"], as_index=False).count()
# ff_transformed_progression_nonrenewable_grouped

# format DF
ff_transformed_progression_nonrenewable_grouped = ff_transformed_progression_nonrenewable_grouped[["State", "Renewable"]]
ff_transformed_progression_nonrenewable_grouped.rename(columns={'Renewable': 'Non_Renewable_Energy_Plants_2014_2015'}, inplace=True)
ff_transformed_progression_nonrenewable_grouped

,State,Non_Renewable_Energy_Plants_2014_2015
0,NSW,32
1,NT,13
2,QLD,30
3,SA,13
4,TAS,2
5,VIC,17
6,WA,63


In [61]:
energy_progression_merged = pd.merge(ff_transformed_progression_renewable_grouped, ff_transformed_progression_nonrenewable_grouped, how="left", on=["State", "State"])
# energy_progression_merged

In [62]:
se_transformed_progression = se_transformed[["State", "Year", "Renewable"]]
se_transformed_progression.head(2)

,State,Year,Renewable
0,NSW,2017-2018,True
1,VIC,2017-2018,True


In [63]:
se_transformed_progression_renewable = se_transformed_progression.loc[se_transformed_progression["Renewable"] == True]
# se_transformed_progression_renewable

# group dataframe by State
se_transformed_progression_renewable_grouped = se_transformed_progression_renewable.groupby(["State"], as_index=False).count()
# se_transformed_progression_renewable_grouped

# format DF
se_transformed_progression_renewable_grouped = se_transformed_progression_renewable_grouped[["State", "Renewable"]]
se_transformed_progression_renewable_grouped.rename(columns={'Renewable': 'Renewable_Energy_Plants_2017_2018'}, inplace=True)
se_transformed_progression_renewable_grouped

,State,Renewable_Energy_Plants_2017_2018
0,ACT,3
1,NSW,45
2,NT,2
3,QLD,14
4,SA,12
5,TAS,28
6,VIC,40
7,WA,20


In [64]:
energy_progression_merged = pd.merge(energy_progression_merged, se_transformed_progression_renewable_grouped, how="left", on=["State", "State"])
# energy_progression_merged

In [65]:
se_transformed_progression_nonrenewable = se_transformed_progression.loc[se_transformed_progression["Renewable"] == False]
# se_transformed_progression_nonrenewable

# group dataframe by State
se_transformed_progression_nonrenewable_grouped = se_transformed_progression_nonrenewable.groupby(["State"], as_index=False).count()
# se_transformed_progression_nonrenewable_grouped

# format DF
se_transformed_progression_nonrenewable_grouped = se_transformed_progression_nonrenewable_grouped[["State", "Renewable"]]
se_transformed_progression_nonrenewable_grouped.rename(columns={'Renewable': 'Non_Renewable_Energy_Plants_2017_2018'}, inplace=True)
se_transformed_progression_nonrenewable_grouped

,State,Non_Renewable_Energy_Plants_2017_2018
0,NSW,28
1,NT,64
2,QLD,32
3,SA,12
4,VIC,14
5,WA,71


In [66]:
energy_progression_merged = pd.merge(energy_progression_merged, se_transformed_progression_nonrenewable_grouped, how="left", on=["State", "State"])
# energy_progression_merged

In [67]:
tt_transformed_progression = tt_transformed[["State", "Year", "Renewable"]]
tt_transformed_progression.head(2)

,State,Year,Renewable
0,NSW,2020-2021,True
1,VIC,2020-2021,True


In [68]:
tt_transformed_progression_renewable = tt_transformed_progression.loc[tt_transformed_progression["Renewable"] == True]
# tt_transformed_progression_renewable

# group dataframe by State
tt_transformed_progression_renewable_grouped = tt_transformed_progression_renewable.groupby(["State"], as_index=False).count()
# tt_transformed_progression_renewable_grouped

# format DF
tt_transformed_progression_renewable_grouped = tt_transformed_progression_renewable_grouped[["State", "Renewable"]]
tt_transformed_progression_renewable_grouped.rename(columns={'Renewable': 'Renewable_Energy_Plants_2020_2021'}, inplace=True)
tt_transformed_progression_renewable_grouped

,State,Renewable_Energy_Plants_2020_2021
0,ACT,5
1,NSW,64
2,NT,3
3,QLD,46
4,SA,16
5,TAS,30
6,VIC,55
7,WA,22


In [69]:
energy_progression_merged = pd.merge(energy_progression_merged, tt_transformed_progression_renewable_grouped, how="left", on=["State", "State"])
# energy_progression_merged

In [70]:
tt_transformed_progression_nonrenewable = tt_transformed_progression.loc[tt_transformed_progression["Renewable"] == False]
# tt_transformed_progression_nonrenewable

# group dataframe by State
tt_transformed_progression_nonrenewable_grouped = tt_transformed_progression_nonrenewable.groupby(["State"], as_index=False).count()
# tt_transformed_progression_nonrenewable_grouped

# format DF
tt_transformed_progression_nonrenewable_grouped = tt_transformed_progression_nonrenewable_grouped[["State", "Renewable"]]
tt_transformed_progression_nonrenewable_grouped.rename(columns={'Renewable': 'Non_Renewable_Energy_Plants_2020_2021'}, inplace=True)
tt_transformed_progression_nonrenewable_grouped

,State,Non_Renewable_Energy_Plants_2020_2021
0,NSW,22
1,NT,63
2,QLD,65
3,SA,15
4,VIC,14
5,WA,71


In [71]:
energy_progression_merged = pd.merge(energy_progression_merged, tt_transformed_progression_nonrenewable_grouped, how="left", on=["State", "State"])
energy_progression_merged

,State,Renewable_Energy_Plants_2014_2015,Non_Renewable_Energy_Plants_2014_2015,Renewable_Energy_Plants_2017_2018,Non_Renewable_Energy_Plants_2017_2018,Renewable_Energy_Plants_2020_2021,Non_Renewable_Energy_Plants_2020_2021
0,ACT,4,NaN,3,NaN,5,NaN
1,NSW,38,32.0,45,28.0,64,22.0
2,NT,2,13.0,2,64.0,3,63.0
3,QLD,14,30.0,14,32.0,46,65.0
4,SA,10,13.0,12,12.0,16,15.0
5,TAS,26,2.0,28,NaN,30,NaN
6,VIC,37,17.0,40,14.0,55,14.0
7,WA,15,63.0,20,71.0,22,71.0


In [ ]:
# import pymongo

# # Setup connection to mongodb
# conn = 'mongodb://localhost:27017'
# client = pymongo.MongoClient(conn)

# # Create database and collection to use
# db = client['energy_db']

# energy_progression_merged = db.energy_progression_merged

# energy_progression_merged.insert_many(energy_progression_merged.to_dict('records'))

Which state produces the most energy per capita? which state produces the least? Divided into renewable vs non renewable.

In [72]:
# Read in CSV
population_summary = pd.read_csv("state_population_summary.csv")

# format DF
population_summary.rename(columns={"Unnamed: 0": "State", "Total": "Population_Total"}, inplace=True)
population_summary = population_summary[["State", "Population_Total"]]

# Removing the Australia(b) total line in index 8
population_summary = population_summary.drop(8)

population_summary

,State,Population_Total
0,New South Wales,8072163
1,Victoria,6503491
2,Queensland,5156138
3,South Australia,1781516
4,Western Australia,2660026
5,Tasmania,557571
6,Northern Territory,232605
7,Australian Capital Territory,454499


In [73]:
population_summary.at[0, 'State'] = "NSW"
population_summary.at[1, 'State'] = "VIC"
population_summary.at[2, 'State'] = "QLD"
population_summary.at[3, 'State'] = "SA"
population_summary.at[4, 'State'] = "WA"
population_summary.at[5, 'State'] = "TAS"
population_summary.at[6, 'State'] = "NT"
population_summary.at[7, 'State'] = "ACT"
population_summary

,State,Population_Total
0,NSW,8072163
1,VIC,6503491
2,QLD,5156138
3,SA,1781516
4,WA,2660026
5,TAS,557571
6,NT,232605
7,ACT,454499


In [ ]:
# population_summary["State"].replace("New South Wales", "NSW", regex=True)
# population_summary

In [ ]:
# # you can do multiple replacements in within one call of the replace method by creating a mapping dictionary
# # very scalable woo
# replacement_mapping_dict = {
#     "New South Wales": "NSW",
#     "Victoria": "VIC",
#     "Queensland": "QLD",
#     "South Australia": "SA",
#     "Western Australia": "WA",
#     "Tasmania": "TAS",
#     "Northern Territory": "NT",
#     "Australian Capital Territory": "ACT",
# }
# population_summary["State"].replace(replacement_mapping_dict, regex=True)

# population_summary

In [ ]:
# energy_per_capita = state_population_summary.csv

In [74]:
tt_eppc_renewable.replace(',','', regex=True, inplace=True)
tt_eppc_renewable = tt_eppc_renewable.fillna(0)

In [76]:
tt_eppc_renewable.head()

0    159867
1    282916
2    649896
3    163293
4      2011
Name: Electricity_Production_MWh, dtype: int64

In [75]:
tt_eppc_renewable = pd.to_numeric(tt_eppc_renewable["Electricity_Production_MWh"])

KeyError: 'Electricity_Production_MWh'

In [ ]:
tt_eppc_renewable.head()


In [ ]:
tt_eppc_renewable['Electricity_Production_MWh'].valuecounts()

In [ ]:
tt_eppc_renewable["Electricity_Production_MWh"] = (tt_eppc_renewable["Electricity_Production_MWh"]/1000000000)
tt_eppc_renewable.head()

In [ ]:
tt_eppc = tt_transformed[["State", "Electricity_Production_MWh", "Renewable"]]
# tt_eppc

tt_eppc_renewable = tt_eppc.loc[tt_eppc["Renewable"] == True]
# tt_eppc_renewable

tt_eppc_renewable = tt_eppc_renewable[["State", "Electricity_Production_MWh"]]

# group dataframe by State
tt_eppc_renewable_grouped = tt_eppc_renewable.groupby(["State"], as_index=False).sum()
tt_eppc_renewable_grouped

# format DF
tt_eppc_renewable_grouped.rename(columns={'Electricity_Production_MWh': 'Renewable_Electricity_Production_MWh'}, inplace=True)
tt_eppc_renewable_grouped

In [ ]:
tt_eppc = tt_transformed[["State", "Electricity_Production_MWh", "Renewable"]]
# tt_eppc

tt_eppc_non_renewable = tt_eppc.loc[tt_eppc["Renewable"] == False]
# tt_eppc_non_renewable

tt_eppc_non_renewable = tt_eppc_non_renewable[["State", "Electricity_Production_MWh"]]

# group dataframe by State
tt_eppc_non_renewable_grouped = tt_eppc_non_renewable.groupby(["State"], as_index=False).sum()
tt_eppc_non_renewable_grouped

# format DF
tt_eppc_non_renewable_grouped.rename(columns={'Electricity_Production_MWh': 'Non_Renewable_Electricity_Production_MWh'}, inplace=True)
tt_eppc_non_renewable_grouped

In [ ]:
eppc_merged = pd.merge(tt_eppc_renewable_grouped, tt_eppc_non_renewable_grouped, how="left", on=["State", "State"])
eppc_merged

In [ ]:
eppc_merged = pd.merge(eppc_merged, population_summary, how="left", on=["State", "State"])
eppc_merged

In [ ]:
eppc_merged.replace(',','', regex=True, inplace=True)
eppc_merged = eppc_merged.fillna(0)
eppc_merged

In [ ]:
eppc_merged["Renewable_Electricity_Production_MWh"] = pd.to_numeric(eppc_merged["Renewable_Electricity_Production_MWh"])

In [ ]:
# create a column for "Per Student Budget" in the school_summary_merge dataframe
eppc_merged['Renewable_MWh_Per_Capita'] = eppc_merged['Renewable_Electricity_Production_MWh'] / eppc_merged['Population_Total'] 
eppc_merged['Non_Renewable_MWh_Per_Capita'] = eppc_merged['Non_Renewable_Electricity_Production_MWh'] / eppc_merged['Population_Total'] 

eppc_merged